In [50]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import random

# prendre une cartede plus haute resolution pour plus de pays
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) 
climate_raster = 'climate_map.tif'


In [51]:
# Define the climate legend as a dictionary mapping RGB values to climate classes
climate_legend = {
    (0, 0, 255): "Af",      # Tropical, rainforest
    (0, 120, 255): "Am",    # Tropical, monsoon
    (70, 170, 250): "Aw",   # Tropical, savannah
    (255, 0, 0): "BWh",     # Arid, desert, hot
    (255, 150, 150): "BWk", # Arid, desert, cold
    (245, 165, 0): "BSh",   # Arid, steppe, hot
    (255, 220, 100): "BSk", # Arid, steppe, cold
    (255, 255, 0): "Csa",   # Temperate, dry summer, hot summer
    (200, 200, 0): "Csb",   # Temperate, dry summer, warm summer
    (150, 150, 0): "Csc",   # Temperate, dry summer, cold summer
    (150, 255, 150): "Cwa", # Temperate, dry winter, hot summer
    (100, 200, 100): "Cwb", # Temperate, dry winter, warm summer
    (50, 150, 50): "Cwc",   # Temperate, dry winter, cold summer
    (200, 255, 80): "Cfa",  # Temperate, no dry season, hot summer
    (100, 255, 80): "Cfb",  # Temperate, no dry season, warm summer
    (50, 200, 0): "Cfc",    # Temperate, no dry season, cold summer
    (255, 0, 255): "Dsa",   # Cold, dry summer, hot summer
    (200, 0, 200): "Dsb",   # Cold, dry summer, warm summer
    (150, 50, 150): "Dsc",  # Cold, dry summer, cold summer
    (150, 100, 150): "Dsd", # Cold, dry summer, very cold winter
    (170, 175, 255): "Dwa", # Cold, dry winter, hot summer
    (90, 120, 220): "Dwb",  # Cold, dry winter, warm summer
    (75, 80, 180): "Dwc",   # Cold, dry winter, cold summer
    (50, 0, 135): "Dwd",    # Cold, dry winter, very cold winter
    (0, 255, 255): "Dfa",   # Cold, no dry season, hot summer
    (55, 200, 255): "Dfb",  # Cold, no dry season, warm summer
    (0, 125, 125): "Dfc",   # Cold, no dry season, cold summer
    (0, 70, 95): "Dfd",     # Cold, no dry season, very cold winter
    (178, 178, 178): "ET",  # Polar, tundra
    (102, 102, 102): "EF"   # Polar, frost
}


In [ ]:
import rasterio
from shapely.geometry import Point
import random

# Function to check if a point is within the bounds of the climate raster
def is_within_raster(lat, lon, raster_bounds):
    min_lon, min_lat, max_lon, max_lat = raster_bounds
    return min_lon <= lon <= max_lon and min_lat <= lat <= max_lat

# Function to check if a point corresponds to valid data in the raster (not NoData)
def has_valid_data(lat, lon, raster, transform, nodata_value):
    # Convert lat/lon to row/col in the raster
    col, row = ~transform * (lon, lat)
    row, col = int(row), int(col)

    # Ensure the coordinates are within the raster's bounds
    if row < 0 or col < 0 or row >= raster.shape[1] or col >= raster.shape[2]:
        return False  # Out of bounds

    # Check the raster value at this row/col to ensure it isn't NoData
    pixel_value = raster[:, row, col]
    if pixel_value[0] == nodata_value:  # Compare with the nodata value
        return False  # NoData point

    return True

# Function to generate 3 valid points (centroid + 2 valid points) per country
def generate_valid_points_for_country(country_geom, climate_raster_file, max_attempts=100):
    points = []

    # Open the climate GeoTIFF file to get bounds, transformation data, and nodata value
    with rasterio.open(climate_raster_file) as src:
        transform = src.transform
        raster_bounds = src.bounds
        climate_data = src.read()  # Read all bands (for RGB values)
        nodata_value = src.nodata  # Access the nodata value from the file

        # Step 1: Add the centroid of the country as the first point
        centroid = country_geom.centroid
        if is_within_raster(centroid.y, centroid.x, raster_bounds) and has_valid_data(centroid.y, centroid.x, climate_data, transform, nodata_value):
            points.append((centroid.y, centroid.x))  # Latitude, Longitude
        else:
            # If the centroid is out of bounds or NoData, generate a valid point inside the country's boundary
            points.append(generate_random_point_within_country(country_geom, climate_data, transform, raster_bounds, nodata_value))

        # Step 2: Generate two additional valid points inside the country and within the GeoTIFF map
        for _ in range(2):
            point = generate_random_point_within_country(country_geom, climate_data, transform, raster_bounds, nodata_value, max_attempts)
            if point is not None:
                points.append(point)
            else:
                points.append(('No valid point found', ''))  # Fallback if no valid points found
    
    return points

# Function to generate a valid random point inside the country's boundaries and within the GeoTIFF map bounds
def generate_random_point_within_country(country_geom, climate_data, transform, raster_bounds, nodata_value, max_attempts=100):
    minx, miny, maxx, maxy = country_geom.bounds
    attempts = 0

    while attempts < max_attempts:
        # Generate a random lat/lon point within the bounding box of the country
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        
        # Check if the point is inside the country and within the bounds of the raster, and has valid data
        if country_geom.contains(random_point) and is_within_raster(random_point.y, random_point.x, raster_bounds):
            if has_valid_data(random_point.y, random_point.x, climate_data, transform, nodata_value):
                return (random_point.y, random_point.x)  # Return valid latitude and longitude
        
        attempts += 1

    # If no valid point is found after max_attempts, return None (or handle as needed)
    return None

country_points = {}

# Iterate through each country and generate points
for _, row in world.iterrows():
    country_name = row['name']
    country_geom = row['geometry']
    points = generate_valid_points_for_country(country_geom, climate_raster)
    country_points[country_name] = points


In [48]:
country_points

{'Fiji': [(-17.31630942638265, 163.85316464458234),
  None,
  (-16.68937739873093, 179.4940876865723)],
 'Tanzania': [(-6.257732428506092, 34.75298985475595),
  (-4.2630597701755875, 34.08615503891083),
  (-2.6197308134879957, 34.449665828344365)],
 'W. Sahara': [(24.291172960208623, -12.13783111160779),
  (26.596919593339038, -9.800409280440071),
  (23.151140023031687, -13.639810704888728)],
 'Canada': [(61.46907614534896, -98.14238137209708),
  (82.58596272277144, -84.30055273668441),
  (58.26212367553686, -63.35221397945749)],
 'United States of America': [(45.70562800215178, -112.5994359115045),
  (65.40840842295935, -146.16216108879448),
  (32.628553393687255, -93.87682836741232)],
 'Kazakhstan': [(48.19166075218232, 67.2846109811001),
  (47.47350056596062, 51.337971701420514),
  (53.884492911493936, 62.207334545857094)],
 'Uzbekistan': [(41.748602664652246, 63.20363952823182),
  (40.86284694517233, 63.06296446004664),
  (43.34384938080679, 57.945712391605625)],
 'Papua New Guinea

In [49]:
def get_climate(lat, lon, raster, transform, raster_bounds, legend):
    min_lon, min_lat, max_lon, max_lat = raster_bounds
    # Check if lat/lon is within raster bounds
    if min_lon <= lon <= max_lon and min_lat <= lat <= max_lat:
        # Convert lat/lon to row/col in the raster
        row, col = ~transform * (lon, lat)  # lon, lat order for transformation
        row, col = int(row), int(col)
        
        # Ensure row/col are within the valid range of the raster's grid
        if 0 <= row < raster.shape[0] and 0 <= col < raster.shape[1]:
            climate_value = raster[row, col]  # Get the numeric climate classification


            
            # Map the numeric value to its Köppen-Geiger class using the legend
            #climate_type = legend.get(climate_value, 'Unknown Climate')

            
            climate_type = climate_value
        else:
            climate_type = 'No Climate Data'
    else:
        climate_type = 'No Climate Data'  # Handle out-of-bounds case
    
    return climate_type
    
def assign_climate_to_countries(country_dict, climate_raster_file, legend):
    country_climate_map = {}

    with rasterio.open(climate_raster_file) as src:
        climate_data = src.read(1)  # Read the first band of the GeoTIFF
        transform = src.transform
        raster_bounds = src.bounds

        for country, points in country_dict.items():
            if points:  # Ensure points are not None or empty
                climate_types = []
                for point in points:
                    if point:  # Ensure each point is valid (not None)
                        lat, lon = point
                        # Get the climate type for each point using the legend
                        climate_type = get_climate(lat, lon, climate_data, transform, raster_bounds, legend)
                        climate_types.append(climate_type)
                    else:
                        climate_types.append('No Point Data')
                country_climate_map[country] = climate_types
            else:
                country_climate_map[country] = ['No Points Generated']

    return country_climate_map

# Now call the function, passing the climate legend
country_climate_map = assign_climate_to_countries(country_points, climate_raster, climate_legend)


# Step 7: Display the results
for country, climates in country_climate_map.items():
    print(f"Country: {country}")
    for i, climate in enumerate(climates):
        print(f"  Point {i+1}: Climate Type: {climate}")

Country: Fiji
  Point 1: Climate Type: No Climate Data
  Point 2: Climate Type: No Point Data
  Point 3: Climate Type: No Climate Data
Country: Tanzania
  Point 1: Climate Type: No Climate Data
  Point 2: Climate Type: No Climate Data
  Point 3: Climate Type: No Climate Data
Country: W. Sahara
  Point 1: Climate Type: 30
  Point 2: Climate Type: 30
  Point 3: Climate Type: 30
Country: Canada
  Point 1: Climate Type: 0
  Point 2: Climate Type: 0
  Point 3: Climate Type: 0
Country: United States of America
  Point 1: Climate Type: 0
  Point 2: Climate Type: 0
  Point 3: Climate Type: 0
Country: Kazakhstan
  Point 1: Climate Type: No Climate Data
  Point 2: Climate Type: No Climate Data
  Point 3: Climate Type: No Climate Data
Country: Uzbekistan
  Point 1: Climate Type: No Climate Data
  Point 2: Climate Type: No Climate Data
  Point 3: Climate Type: No Climate Data
Country: Papua New Guinea
  Point 1: Climate Type: No Climate Data
  Point 2: Climate Type: No Climate Data
  Point 3: Clim